# DATA LOAD

Read the original Excel information and transform it to be processed.

The original information is the folder 'data/original', classified by years and in some file by year

In every file exists multiple sheet, some of them with userful information and others that want be used.

The files and sheets will be detected using the cell headers. 

The preprocessed information will be stored in folder 'data/preprocessed', classified by language.


In [1]:
import json
import os.path
import sys
import csv
import numpy as np
import pandas as pd
from colorama import Fore

In [2]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path

In [13]:
import spacy
from spacy_langdetect import LanguageDetector

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")
# ca_fasttext_wiki
# es_core_news_sm
# en_core_web_sm

nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [4]:
pathori = "../data/original"
pathdest = "../data/preprocessed/"
debug = True

In [5]:
def delete_preprocessed_files():
    
    languages = ["ca", "es", "en", "xx"]
    for language in languages:
        url = pathdest + "comentaris_" + language + ".csv"
        if os.path.exists(url):
            os.remove(url)
            if debug:
                print ("File deleted: " + url)
    
    

In [6]:
# Test if a file has the correct format to load his information

def validateSheet(data):

    cols = data.columns
    if (cols.size > 5):
        if (('Codi pregunta' in cols) or ('Comentari' in cols) or ('Tipus incidencia' in cols)):
            if (('Codi pregunta' in cols) and ('Comentari' in cols) and ('Tipus incidencia' in cols)):
                print (Fore.GREEN + " " * 8  + "VALID")
                return True
            else:
                print (Fore.RED + " " * 8 + "WARNING (see column names)")
                print (cols)
                return False
        else:
            print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")
            return False
    else:
        print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")
        return False


In [7]:
# Reorganize information to be exported in preprocessed files

def process_file (data_ori):
# folder = "17-18"
# file = "assignatura_professor_comentaris_LS_17-18_1s.xlsx"
# sheet = "Hoja1"
# xlsx = pd.ExcelFile(pathori+"/"+folder+"/"+file)
# data_ori = pd.read_excel(xlsx, sheet)

#    print(Fore.BLACK + "Comments:" + data.shape[0])

    data_ori["TipusPregunta"] = data_ori["Codi pregunta"].str[0]
    data_ori["TipusIncidencia"] = data_ori["Tipus incidencia"]
    
# Obtain language of comments
    data_ori["Idioma"] = ""
    for x in range(data_ori.shape[0]):
        comment = data_ori.loc[x]["Comentari"]
        data_ori.loc[x, "Idioma"] = nlp(comment)._.language["language"]

#    data2 = data.rename(columns={"Tipus incidencia":"TipusIncidencia"})
    data_res = data_ori[["Curs","TipusPregunta","TipusIncidencia","Comentari","Idioma"]]
    
    if debug:
#        print("Comments:"  + data_res.shape[0])
        print(data_res[data_res["TipusIncidencia"].notnull()].head())
    
    return data_res

In [8]:
# Load data from file

def load_file(url):
    try:
        if debug:
            print (Fore.BLACK + "File:   " + url)
        sheet = ""
        xlsx = pd.ExcelFile(url)
        for sheet in xlsx.sheet_names:
            if debug:
                print (Fore.BLACK + "Sheet:  " + sheet)
            data = pd.read_excel(xlsx, sheet)
            if validateSheet(data):
#                print ("Processing file: " + url)
                data_res = process_file(data)
                write_file(data_res)
    
    except:
        print (Fore.LIGHTWHITE_EX + " " * 8 + "NO VALID")

In [9]:
# Load data from folder

def load_folder(folder):
    
    if debug:
        print (Fore.BLACK + 'Folder: ' + folder)
    if os.path.isdir(folder):
        for file in os.listdir(folder):
            url = folder+"/"+file
#            print (url)
            if os.path.isdir(url):
                load_folder(url)
            if os.path.isfile(url):
                load_file(url)


In [10]:
# Write data preprocessed into disk, in one file by language

def write_file(data_res):
    print ("write_file")
        
    if debug:
        print(data_res.value_counts("Idioma"))
    
    # Official languages
    languages = ["ca", "es", "en"]
    for language in languages:
        url = "comentaris_" + language + ".csv"
        data_lang = data_res[data_res["Idioma"]==language]
        
        if os.path.exists(pathdest + url):
            data_lang.to_csv(pathdest + url, index=False, mode="a", header=False)
            if debug:            
                print ("File updated: " + url)
        else:
            data_lang.to_csv(pathdest + url, index=False, mode="w", header=True)
            if debug:
                print ("File created: " + url)
            
    # Other languages        
    url = "comentaris_xx.csv"
    data_lang = data_res[~data_res["Idioma"].isin(languages)]
    if os.path.exists(pathdest + url):
        data_lang.to_csv(pathdest + url, index=False, mode="a", header=False)
        if debug:            
            print ("File updated: " + url)        
    else:
        data_lang.to_csv(pathdest + url, index=False, mode="w", header=True)
        if debug:
            print ("File created: " + url)
    
    if first_file:
        first_file = False
        

In [11]:
def analyze_files():
    
    if debug:
        print (Fore.BLACK + "Preprocessed file analysis:")

#   Load preprocessed files
    data_total = pd.DataFrame(columns=["Curs","TipusPregunta","TipusIncidencia","Idioma","File","Comentari"])

    languages = ["ca","es","en","xx"]
    for language in languages:
        file = "comentaris_" + language + ".csv"
        data_lang = pd.read_csv(pathdest + file)

        if debug:
            print ("* " + file + ": ", end="")
            print (data_lang.shape[0], end="")
            print (" registers")

        data_lang["Fitxer"] = language
        data_lang = data_lang.groupby(["Curs","TipusPregunta","TipusIncidencia","Idioma","Fitxer"], dropna=False).count().reset_index()
        data_total = pd.concat([data_total, data_lang])

    data_total = data_total.reset_index()
    display (data_total)

#   Distribution by Curs    
    data_group = data_total[["Curs","Fitxer","Comentari"]].groupby(["Curs","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="Curs", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)
        
#   Distribution by Tipus Pregunta        
    data_group = data_total[["TipusPregunta","Fitxer","Comentari"]].groupby(["TipusPregunta","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="TipusPregunta", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)
        
#   Distribution by Tipus Incidencia        
    data_group = data_total[["TipusIncidencia","Fitxer","Comentari"]].groupby(["TipusIncidencia","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="TipusIncidencia", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)

#   Distribution by Idioma        
    data_group = data_total[["Idioma","Fitxer","Comentari"]].groupby(["Idioma","Fitxer"]).sum().reset_index()
    data_pivot = data_group.pivot(index="Idioma", columns="Fitxer", values="Comentari")
    if debug:
        display (data_pivot)
        

In [14]:
delete_preprocessed_files()
load_folder(pathori)
analyze_files()
print (Fore.BLACK + "ORIGINAL FILES PREPROCESSED")

File deleted: ../data/preprocessed/comentaris_ca.csv
File deleted: ../data/preprocessed/comentaris_es.csv
File deleted: ../data/preprocessed/comentaris_en.csv
File deleted: ../data/preprocessed/comentaris_xx.csv
Folder: ../data/original
Folder: ../data/original/17-18
File:   ../data/original/17-18/assignatura_professor_comentaris_LS_17-18_1s.xlsx
Sheet:  Hoja1
        VALID
       Curs TipusPregunta                      TipusIncidencia  \
20  2017-18             P              Comentari d'assignatura   
24  2017-18             P  No ha impartit classe a aquest grup   
25  2017-18             P  No ha impartit classe a aquest grup   
27  2017-18             P  No ha impartit classe a aquest grup   
29  2017-18             P  No ha impartit classe a aquest grup   

                                            Comentari Idioma  
20  Módulo 1 áreas para mejorar: \r\nLa tarea nr. ...     es  
24                              No he tenido contacto     es  
25                               No h

       Curs TipusPregunta        TipusIncidencia  \
3   2019-20             P  Comentari problemàtic   
10  2019-20             A      Comentari ofensiu   
49  2019-20             P  Comentari problemàtic   
85  2019-20             P  Comentari problemàtic   
87  2019-20             P  Comentari problemàtic   

                                            Comentari Idioma  
3   1r. Que d'una vegada faci les classes als hora...     ca  
10  Los exámenes han ido a putear 50min 45 pregunt...     es  
49  Arribava bastant tard habitualment. Trobo norm...     ca  
85  Era impossible contactar amb ell, tant fos per...     ca  
87  Presencialment, no respectava que els alumnes ...     ca  
write_file
Idioma
ca         573
es         108
en          11
it           9
pt           7
fr           6
de           2
id           1
UNKNOWN      1
dtype: int64
File updated: comentaris_ca.csv
File updated: comentaris_es.csv
File updated: comentaris_en.csv
File updated: comentaris_xx.csv
        NO VALI

,index,Curs,TipusPregunta,TipusIncidencia,Idioma,File,Comentari,Fitxer
0,0,2017-18,A,Comentari de professor,ca,NaN,2,ca
1,1,2017-18,A,NaN,ca,NaN,11,ca
2,2,2017-18,P,Comentari d'assignatura,ca,NaN,1,ca
3,3,2017-18,P,NaN,ca,NaN,13,ca
4,4,2018-19,A,Comentari de professor,ca,NaN,18,ca
...,...,...,...,...,...,...,...,...
119,50,2019-20,P,NaN,pt,NaN,8,xx
120,51,2019-20,P,NaN,ro,NaN,3,xx
121,52,2019-20,P,NaN,sl,NaN,1,xx
122,53,2019-20,P,NaN,so,NaN,1,xx


Fitxer,ca,en,es,xx
Curs,,,,
2017-18,27,2,55,6
2018-19,875,251,294,48
2019-20,5093,77,996,131


Fitxer,ca,en,es,xx
TipusPregunta,,,,
A,2619.0,97.0,624.0,59.0
P,3376.0,233.0,721.0,126.0
ca,0.0,NaN,NaN,NaN


Fitxer,ca,en,es,xx
TipusIncidencia,,,,
Canvi de professor,15.0,NaN,2.0,NaN
Comentari d'assignatura,3.0,1.0,3.0,NaN
Comentari de professor,37.0,2.0,13.0,1.0
Comentari excel·lent,1.0,2.0,1.0,NaN
Comentari ofensiu,7.0,NaN,3.0,1.0
Comentari problemàtic,224.0,NaN,39.0,1.0
Duplicar a professor,2.0,NaN,3.0,NaN
Exclamacions o emoticones excessius,1.0,NaN,NaN,1.0
Faltes d'ortografia,14.0,NaN,4.0,NaN


Fitxer,ca,en,es,xx
Idioma,,,,
UNKNOWN,NaN,NaN,NaN,10.0
af,NaN,NaN,NaN,2.0
ca,5991.0,NaN,NaN,NaN
cy,NaN,NaN,NaN,1.0
de,NaN,NaN,NaN,8.0
en,NaN,330.0,NaN,NaN
es,NaN,NaN,1345.0,NaN
et,NaN,NaN,NaN,2.0
fi,NaN,NaN,NaN,2.0


ORIGINAL FILES PREPROCESSED
